In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-26"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11829,2024-12-26,Europa Liga Bnxt,16:30,Circus Brussels,Spirou Charleroi,1.59,2.15,151.5,1.88,1.79,-4.5,2.05,1.66,48RGEmGj,0.628931,0.465116,0.531915,0.558659,0.094047,148.014,102.937836,0.695460,1.2,1.643168,1.369306,81.37,1.862,1.128769,0.606213,-5.0,124.550,41.312277,0.331692,0.6,1.341641,2.236068,105.30,1.664,0.682188,0.409969,-7.0,79,78,1.03,1.35,91.518,142.286,0.211754,0.034681,0.148664,0.22,0.044,13.409091,0.814690,0.8,-0.014690,-1.81,-0.362,-3.176796,0.527787,0.5,-0.027787
11830,2024-12-26,China Cba,08:35,Zhejiang Guangsha,Sichuan,1.03,8.30,199.5,1.80,1.86,-19.5,2.03,1.66,0bJuhxUP,0.970874,0.120482,0.555556,0.537634,0.091356,108.310,13.449011,0.124171,2.4,1.341641,0.559017,121.20,1.114,0.070569,0.063348,44.0,655.948,624.615674,0.952234,1.2,1.643168,1.369306,133.65,5.864,4.808454,0.819996,-18.0,101,99,1.20,1.35,126.024,683.970,1.101965,0.023184,0.141805,1.28,0.256,0.117188,0.812141,1.0,0.187859,-5.00,-1.000,-7.300000,0.204538,0.1,-0.104538
11831,2024-12-26,China Wcba Feminina,08:30,Liaoning F,Hefei F,1.57,2.20,146.5,1.80,1.86,-8.5,1.95,1.72,C43iOIx3,0.636943,0.454545,0.555556,0.537634,0.091488,157.878,50.798697,0.321759,0.6,1.341641,2.236068,126.75,2.232,0.617471,0.276645,-34.0,552.272,784.823459,1.421081,1.8,1.643168,0.912871,1953.00,6.302,8.234972,1.306724,37.0,65,93,1.95,21.00,298.960,0.000,0.236327,0.023184,0.088629,-2.27,-0.454,-1.255507,0.425502,0.4,-0.025502,0.00,0.000,inf,0.000000,0.0,0.000000
11832,2024-12-26,China Wcba Feminina,08:30,Wuhan F,Henan F,1.42,2.56,153.5,1.81,1.85,-10.5,2.01,1.68,pbB50Fpd,0.704225,0.390625,0.552486,0.540541,0.094850,91.528,14.431664,0.157675,3.0,0.000000,0.000000,90.72,1.072,0.081670,0.076185,89.0,98.440,10.377974,0.105424,2.4,1.341641,0.559017,103.79,1.142,0.143422,0.125589,72.0,84,97,1.08,1.07,286.324,444.412,0.405076,0.015456,0.126474,-0.95,-0.190,-2.210526,0.464065,0.4,-0.064065,-1.91,-0.382,-4.083770,0.295210,0.1,-0.195210
11833,2024-12-26,Coreia Do Sul Kbl,07:00,KoGas,LG Sakers,1.80,1.98,152.5,1.86,1.86,-2.5,2.01,1.72,pzmYWwaD,0.555556,0.505051,0.537634,0.537634,0.060606,136.082,50.160098,0.368602,1.2,1.643168,1.369306,148.77,1.688,0.471932,0.279581,-30.0,132.496,22.979362,0.173434,3.0,0.000000,0.000000,102.12,1.676,0.322847,0.192629,68.0,87,69,1.71,1.48,109.690,155.046,0.067344,0.000000,0.109952,-0.80,-0.160,-5.000000,0.700486,0.6,-0.100486,0.41,0.082,11.951220,0.484575,0.4,-0.084575
11834,2024-12-26,Eslováquia Extraliga,10:00,Inter Bratislava,Prievidza,1.67,2.02,165.5,1.80,1.86,-3.5,2.01,1.68,lhDJdElm,0.598802,0.495050,0.555556,0.537634,0.093852,125.860,23.633553,0.187777,1.2,1.643168,1.369306,128.25,1.468,0.322211,0.219490,-5.0,430.856,672.627420,1.561142,2.4,1.341641,0.559017,1634.00,4.834,7.920336,1.638464,82.0,95,86,1.35,19.00,105.230,458.300,0.134139,0.023184,0.126474,-1.59,-0.318,-2.106918,0.679605,0.7,0.020395,14.00,2.800,0.364286,0.482403,0.2,-0.282403
11835,2024-12-26,Eslováquia Extraliga,14:00,Handlova,Iskra Svit,1.09,5.74,157.5,1.80,1.86,-14.5,1.97,1.71,2ewgBD3t,0.917431,0.174216,0.555556,0.537634,0.091647,141.410,23.061204,0.163080,1.2,1.643168,1.369306,136.01,1.834,0.464898,0.253488,5.0,313.450,178.371019,0.569057,0.6,1.341641,2.236068,142.04,4.218,2.241689,0.531458,-63.0,67,67,2.03,2.12,141.838,361.590,0.962825,0.023184,0.099917,0.91,0.182,0.494505,0.588

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11831,08:30,China Wcba Feminina,Liaoning F,Hefei F,1.57,Back Home
11834,10:00,Eslováquia Extraliga,Inter Bratislava,Prievidza,1.67,Back Home
